# Solving Inverse Problems with NNs

**TODOs: 1) bottom: show targets in results;**


Inverse problems encompass a large class of practical scenarios that appear in science. In general, the goal here is not to directly compute a physical field like the velocity at a future time (this is the typical scenario for a _forward_ solve), but instead more generically compute a parameter in the model equation such that certain constraints are fulfilled. A very common objective is to find the optimal setting for a single parameter given some constraints. E.g., this could be the global diffusion constant for an advection-diffusion model such that it fits measured data as accurately as possible. Inverse problems are encountered for any model parameter adjusted via observations, or the reconstruction of initial conditions, e.g., for particle imaging velocimetry (PIV). More complex cases aim for computing boundary geometries w.r.t. optmal conditions, e.g. to obtain a shape with minimal drag in a fluid flow.

A key aspect demonstrated below will be that we're not aiming for solving only a _single instance_ of an inverse problem, but we'd like to use deep learning to solve a _larger collection_ of inverse problems. Thus, unlike the PINN example of {doc}`physicalloss-code` or the DP optimization of {doc}`diffphys-code-ns`, where we've solved an optimization problem for specific instances of inverse problems, we now aim for training an NN that learns to solve a larger class of inverse problems, i.e., a whole solution manifold. Nonetheless, we of course need to rely on a certain degree of similarity for these problems, otherwise there's nothing to learn (and the implied assumption of continuity in the solution manifold breaks down).

Below we will run a very challenging test case as a representative of these inverse problems: we will aim for computing a high dimensional control function that exerts forces over the full course of an incompressible fluid simulation in order to reach a desired goal state for a passively advected marker in the fluid. This means we only have very indirect constraints to be fulfilled (a single state at the end of a sequence), and a large number of degrees of freedom (the control force function is a space-time function with the same degrees of freedom as the flow field itself).

The _long-term_ nature of the control is one of the aspects which makes this a tough inverse problem: any changes to the state of the physical system can lead to large change later on in time, and hence a controller needs to anticipate how the system will behave when it is influenced. This means an NN also needs to learn how the underlying physics evolve and change, and this is exaclty where the gradients from the DP training come in to guide the learning task towards solution that can reach the goal.


## Formulation

With the notation from {doc}`overview-equations` this gives the minmization problem 

$\text{arg min}_{\theta} \sum_m \sum_i (f(x_{m,i} ; \theta)-y^*_{m,i})^2$

where $y^*_{m,i}$ denotes the samples of the target state of the marker field, 
and $x_{m,i}$ denotes the simulated state of the marker density.
As before, the index $i$ samples our solution at different spatial locations (typically all grid cells), while the index $n$ here indicates a large collection of different target states.


Our goal is to train two networks $\mathrm{OP}$ and $\mathrm{CFE}$ with weights
$\theta_{\mathrm{OP}}$ and $\theta_{\mathrm{CFE}}$ such that a sequence 

$
\newcommand{\pde}{\mathcal{P}}
\newcommand{\net}{\mathrm{CFE}}
\mathbf{u}_{n},d_{n} = \pde(\net(~\pde(\net(\cdots \pde(\net( \mathbf{u}_0,d_0  ))\cdots)))) = (\pde\net)^n ( \mathbf{u}_0,d_0 ) .
$

minimizes the loss above. The $\mathrm{OP}$ network is a predictor that determines the action of the $\mathrm{CFE}$ network given the target $d^*$, i.e., $\mathrm{OP}(\mathbf{u},d,d^*)=d_{OP}$,
and $\mathrm{CFE}$ acts additively on the velocity field via
$\mathrm{CFE}(\mathbf{u},d,d_{OP}) = \mathbf{u} + f_{\mathrm{OP}}(\mathbf{u},d,d_{OP};\theta_{\mathrm{OP}})$,  
where we've used $f_{\mathrm{OP}}$ to denote the NN representation of $\mathrm{CFE}$).

For this problem, the model PDE $\mathcal{P}$ contains a discretized version of the incompressible Navier-Stokes equations in two dimensions for a velocity $\mathbf{u}$:

$\begin{aligned}
  \frac{\partial u_x}{\partial{t}} + \mathbf{u} \cdot \nabla u_x &= - \frac{1}{\rho} \nabla p 
  \\
  \frac{\partial u_y}{\partial{t}} + \mathbf{u} \cdot \nabla u_y &= - \frac{1}{\rho} \nabla p 
  \\
  \text{s.t.} \quad \nabla \cdot \mathbf{u} &= 0,
\end{aligned}$

with an additional transport equation for the marker density $d$:

$\begin{aligned}
  \frac{\partial d}{\partial{t}} + \mathbf{u} \cdot \nabla d &= 0 .
\end{aligned}$

To summarize, we have a predictor $\mathrm{OP}$ that gives us a direction, an actor $\mathrm{CFE}$ that exerts a force on a physical model $\mathcal{P}$. They all need to play hand in hand to reach a given target after $n$ iterations of the simulation. As apparent from this formulation, it's not a simple inverse problem, especially due to the fact that all three functions are non-linear. This is exactly why the gradients from the DP approach are so important.



## Learning to Control Incompressible Fluids 

This notebook will walk you through data generation, supervised network initialization and end-to-end training using our differentiable PDE solver, [Φ<sub>Flow</sub>](https://github.com/tum-pbs/PhiFlow). 
(_Note: this example uses an older version of Φ<sub>Flow</sub> (1.4.1)._)

The code below replicates the shape transitions experiment from the ICLR 2020 paper by Holl et al. {cite}`holl2019pdecontrol`, [Learning to Control PDEs with Differentiable Physics](https://ge.in.tum.de/publications/2020-iclr-holl/), further details can be found in section D.2 of the [appendix](https://openreview.net/pdf?id=HyeSin4FPB).

First we need to load phiflow and download the control code helpers (which will end up under `./src`) and some numpy arrays with intial shapes.

In [7]:
#!pip install --upgrade --quiet git+https://github.com/tum-pbs/PhiFlow@1.4.1

import matplotlib.pyplot as plt
from phi.flow import *

# # this essentially copies over the data and code from https://github.com/holl-/PDE-Control
# if not os.path.isfile('shapes/Shape_000000.npz'):
#   import urllib.request
#   url="https://ge.in.tum.de/download/2020-iclr-holl/control.zip"
#   print("Downloading, this can take a few minutes the first time...")
#   urllib.request.urlretrieve(url, 'control.zip')
#   import os
#   os.system("unzip control.zip")

if not os.path.isdir('PDE-Control'):
  print("Cloning, PDE-Control repo, this can take a moment")
  os.system("git clone --recursive https://github.com/holl-/PDE-Control.git")
    
# now we can load the necessary phiflow libraries and helper functions
import sys; 
#sys.path.append('./src')
sys.path.append('PDE-Control/src')
from shape_utils import load_shapes, distribute_random_shape
from control.pde.incompressible_flow import IncompressibleFluidPDE
from control.control_training import ControlTraining
from control.sequences import StaggeredSequence, RefinedSequence


## Data Generation

Before starting the training, we have to generate a data set to train with. I.e., the goal is to pre-compute our a set of ground truth time sequences $u^*$. Due to the complexity of the training below, we'll use a staged approach that pre-trains a supervised network as a rough initialization, and then refines it to learn control looking further and further ahead into the future (i.e., being trained for longer simulation sequences). 

First, let's set up a domain and basic parameters of the data generation step.

In [8]:
domain = Domain([64, 64])  # 1D Grid resolution and physical size
step_count = 16  # how many solver steps to perform
dt = 1.0  # Time increment per solver step
example_count = 1000
batch_size = 100
data_path = 'shape-transitions'
pretrain_data_path = 'moving-squares'
shape_library = load_shapes('PDE-Control/notebooks/shapes')

The following cell creates the dataset we want to train our model on.
Each example consists of a start and target (end) frame which are generated by placing a random shape somewhere within the domain.

In [9]:
for scene in Scene.list(data_path): scene.remove()

for _ in range(example_count // batch_size):
    scene = Scene.create(data_path, count=batch_size, copy_calling_script=False)
    print(scene)
    start = distribute_random_shape(domain.resolution, batch_size, shape_library)
    end__ = distribute_random_shape(domain.resolution, batch_size, shape_library)
    [scene.write_sim_frame([start], ['density'], frame=f) for f in range(step_count)]
    scene.write_sim_frame([end__], ['density'], frame=step_count)

shape-transitions/sim_000000
shape-transitions/sim_000100
shape-transitions/sim_000200
shape-transitions/sim_000300
shape-transitions/sim_000400
shape-transitions/sim_000500
shape-transitions/sim_000600
shape-transitions/sim_000700
shape-transitions/sim_000800
shape-transitions/sim_000900


Since this dataset does not contain any intermediate frames, it does not allow for supervised pretraining. This is because to pre-train a CFE, two consecutive frames are required and to pretrain an $OP_n$, three frames with distance $n/2$ are needed.

Instead, we create a second dataset which contains intermediate frames. This does not need to look like the actual dataset since it's only used for network initialization. Here, we linearly move a rectangle around the domain.

In [10]:
for scene in Scene.list(pretrain_data_path): scene.remove()

for scene_index in range(example_count // batch_size):
    scene = Scene.create(pretrain_data_path, count=batch_size, copy_calling_script=False)
    print(scene)
    pos0 = np.random.randint(10, 56, (batch_size, 2))  # start position
    pose = np.random.randint(10, 56, (batch_size, 2))  # end position
    size = np.random.randint(6,  10,  (batch_size, 2))
    for frame in range(step_count+1):
        time = frame / float(step_count + 1)
        pos = np.round(pos0 * (1 - time) + pose * time).astype(np.int)
        density = AABox(lower=pos-size//2, upper=pos-size//2+size).value_at(domain.center_points())
        scene.write_sim_frame([density], ['density'], frame=frame)

moving-squares/sim_000000
moving-squares/sim_000100
moving-squares/sim_000200
moving-squares/sim_000300
moving-squares/sim_000400
moving-squares/sim_000500
moving-squares/sim_000600
moving-squares/sim_000700
moving-squares/sim_000800
moving-squares/sim_000900


## Supervised Initialization

In [11]:
test_range = range(100)
val_range = range(100, 200)
train_range = range(200, 1000)

The following cell trains all OP$_n \,\, \forall n\in\{2,4,8,16\}$.
The `ControlTraining` class is used to set up the optimization problem.

The loss for the supervised initialization is defined as the observation loss at the center frame.

$\boldsymbol L_o^\textrm{sup} = \left|\mathrm{OP}(o(t_i),o(t_j)) - u^*\left(\frac{t_i+t_j}{2}\right)\right|^2.$

Consequently, no sequence needs to be simulated (`sequence_class=None`) and an observation loss is required at frame $\frac n 2$ (`obs_loss_frames=[n // 2]`).
The pretrained network checkpoints are stored in `supervised_checkpoints`.

*Note: The next cell will run for some time. If you have a set of pretrained networks, you can skip it and load the pretrained networks instead (see instructions below).*

In [12]:
supervised_checkpoints = {}

for n in [2, 4, 8, 16]:
    app = ControlTraining(n, IncompressibleFluidPDE(domain, dt),
                          datapath=pretrain_data_path, val_range=val_range, train_range=train_range, trace_to_channel=lambda _: 'density',
                          obs_loss_frames=[n//2], trainable_networks=['OP%d' % n],
                          sequence_class=None).prepare()
    for i in range(1000):
        app.progress()  # Run Optimization for one batch
    supervised_checkpoints['OP%d' % n] = app.save_model()


App created. Scene directory is /Users/thuerey/phi/model/control-training/sim_000000 (INFO), 2021-04-07 15:04:48,590n





Sequence class: <class 'control.sequences.SkipSequence'> (INFO), 2021-04-07 15:04:48,733n

Partition length 2 sequence (from 0 to 2) at frame 1

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.


/Users/thuerey/Dropbox/mbaDevelSelected/phiflow-v1.5/phi/struct/context.py:18: UserWarning: struct.unsafe() is deprecated. Use map() with new_type argument to avoid validation.
  warnings.warn("struct.unsafe() is deprecated. Use map() with new_type argument to avoid validation.")


ValueError: Dimensions must be equal, but are 35 and 37 for 'OP2/add_1' (op: 'AddV2') with input shapes: [?,35,35,16], [?,37,37,16].

In [ ]:
supervised_checkpoints

In [ ]:
# supervised_checkpoints = {'OP%d' % n: '../networks/shapes/supervised/OP%d_1000' % n for n in [2, 4, 8, 16]}

## CFE Pretraining with Differentiable Physics

To pretrain the CFE, we set up a simulation with a single step of the differentiable solver.

The following cell trains the CFE network from scratch. You can skip it an load the checkpoint by running the cell after.

In [ ]:
app = ControlTraining(1, IncompressibleFluidPDE(domain, dt),
                      datapath=pretrain_data_path, val_range=val_range, train_range=train_range, trace_to_channel=lambda _: 'density',
                      obs_loss_frames=[1], trainable_networks=['CFE']).prepare()
for i in range(1000):
    app.progress()  # Run Optimization for one batch
supervised_checkpoints['CFE'] = app.save_model()

In [ ]:
# supervised_checkpoints['CFE'] = '../networks/shapes/CFE/CFE_2000'

Note that we have not actually set up a simulation for the training. To evaluate the quality of our current solutions, we can use the `ControlTraining` class again but this time passing a `sequence_class`.
With a sequence class, `ControlTraining` sets up the necessary physics steps and networks.
Example sequence classes are `StaggeredSequence` and `RefinedSequence`.

**TODO** , add preview?


## End-to-end Training with Differentiable Physics

To initiate an end-to-end training of the $CFE$ and all $OP_n$ networks with the differentiable physics loss , we create a new `ControlTraining` instance with the staggered execution scheme.

The following cell builds the computational graph with `step_count` solver steps without initializing the network weights.


In [ ]:
staggered_app = ControlTraining(step_count, IncompressibleFluidPDE(domain, dt),
                                datapath=data_path, val_range=val_range, train_range=train_range, trace_to_channel=lambda _: 'density',
                                obs_loss_frames=[step_count], trainable_networks=['CFE', 'OP2', 'OP4', 'OP8', 'OP16'],
                                sequence_class=StaggeredSequence, learning_rate=5e-4).prepare()

The next cell initializes the networks using the supervised checkpoints and then trains all networks jointly. You can skip it and load the checkpoint by running the cell after. You can increase the number of optimization steps or execute the next cell multiple times to further increase performance.

*Note: The next cell will run for some time. You can skip the next two cells and load the pretrained networks instead.*

In [ ]:
staggered_app.load_checkpoints(supervised_checkpoints)
for i in range(1000):
    staggered_app.progress()  # Run staggered Optimization for one batch
staggered_checkpoint = staggered_app.save_model()

In [ ]:
# staggered_checkpoint = {net: '../networks/shapes/staggered/all_53750' for net in ['CFE', 'OP2', 'OP4', 'OP8', 'OP16']}
# staggered_app.load_checkpoints(staggered_checkpoint)

Now that the network is trained, we can infer some trajectories from the test set.
This corresponds to Fig 5b and 18b from the [paper](https://openreview.net/pdf?id=HyeSin4FPB).

In [ ]:
states = staggered_app.infer_all_frames(test_range)

In [ ]:
import pylab
batches = [0, 1, 2]
pylab.subplots(len(batches), 9, sharey='row', sharex='col', figsize=(12, 7))
pylab.tight_layout(w_pad=0)
for i, batch in enumerate(batches):
    for t in range(9):
        pylab.subplot(len(batches), 9, t + 1 + i * 9)
        pylab.title('t=%d' % t * 2)
        pylab.imshow(states[t * 2].density.data[batch, ..., 0], origin='lower')


**TODO, show targets; discuss**


## Next Steps

- Change the `test_range` indices to look at different examples, and test the generalization of the trained controller networks.
- Try using a `RefinedSequence` (instead of a `StaggeredSequence`) to train with the prediction refinement scheme. This will yield a further improved control.